### Importing usefull Library 

In [1]:
import os
import pandas as pd
import numpy as np
import surprise
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise import KNNWithMeans

In [2]:
os.getcwd()

'D:\\Manipal Data Science\\Term 2\\Machine Learning\\ML Assignmnt\\Assignment3\\ml-latest-small'

In [3]:
path = 'D:\\Manipal Data Science\\Term 2\\Machine Learning\\ML Assignmnt\\Assignment3\\ml-latest-small'
os.chdir(path)

In [4]:
rating = pd.read_csv("ratings.csv")
link = pd.read_csv("links.csv")
movie = pd.read_csv("movies.csv")
tag = pd.read_csv("tags.csv")

In [5]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


### 1.	Create recommender system using ratings.csv file

In [6]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(rating[['userId','movieId','rating']], reader)

In [7]:
[trainset,testset] = train_test_split(data, test_size=0.15, shuffle=True)

recom = KNNWithMeans(k=70, sim_options={'name':'cosine','user_based':True})

recom.fit(data.build_full_trainset())      # using full training set and not doing k

test_pred = recom.test(testset)
RMSE = accuracy.rmse(test_pred)


Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 0.7840


### 2.	Display predicted rating for a particular userID and MovieID combinations (both taken as keyboard input)

In [8]:
userID = int(input("Enter the userID: "))
movieID = int(input("Enter the movieID: "))
print("rating for the userID:",userID, "and",'movieID',movieID ,'is: ',round(recom.predict(userID,movieID)[3],2))

Enter the userID: 6
Enter the movieID: 600
rating for the userID: 6 and movieID 600 is:  3.98


In [9]:
rating.shape,tag.shape,movie.shape,link.shape

((100836, 4), (3683, 4), (9742, 3), (9742, 3))

### 3.	Recommend a user to watch top 10 movies, which has not watched yet (as per his rating predictions) ,threshold rating as 2.5

In [28]:
movie_list = {}
user_id = int(input('Enter user_id: '))
for movie_id in range(1,193609):
    if recom.predict(user_id,movie_id)[3] > 2.5:
        movie_list[movie_id] = recom.predict(user_id,movie_id)[3]

        
from collections import Counter

movie_rating = Counter(movie_list)
high_rated_movie = movie_rating.most_common(10)
print("\n")
print("Ten Suggested movies for user {0} are listed below".format(user_id))


for i in range(10):
    print(i+1,"-->",list(movie[movie.movieId==high_rated_movie[i][0]]["title"])[0])

Enter user_id: 6


Ten Suggested movies for user 6 are listed below
1 --> Denise Calls Up (1995)
2 --> Babes in Toyland (1934)
3 --> On the Beach (1959)
4 --> Bossa Nova (2000)
5 --> Man with the Golden Arm, The (1955)
6 --> Slumber Party Massacre II (1987)
7 --> Slumber Party Massacre III (1990)
8 --> Sorority House Massacre (1986)
9 --> Sorority House Massacre II (1990)
10 --> Two Family House (2000)


### 4.	Display the MovieID, IMDB ID, Average User Rating  genres and tag of all the movies as a data frame.

----

In [29]:
rating.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [31]:
avg_rating = {}
for i in range(1,193609):
    if (i==33)|(i==35)|(i==37):                   #these movieid has no rating
        pass
    else:
        avg_rating[i] = round((rating[rating.movieId == i]['rating'].mean()),2)

In [32]:
df1 = pd.merge(rating[["movieId","rating"]],tag[["movieId","tag"]],how="left",on="movieId")

df2 = pd.merge(df1,link[["movieId","imdbId"]],how="left",on="movieId")
new_df = pd.merge(df2,movie[["movieId","genres"]],how="left",on="movieId")
new_df.head()

,movieId,rating,tag,imdbId,genres
0,1,4.0,pixar,114709,Adventure|Animation|Children|Comedy|Fantasy
1,1,4.0,pixar,114709,Adventure|Animation|Children|Comedy|Fantasy
2,1,4.0,fun,114709,Adventure|Animation|Children|Comedy|Fantasy
3,3,4.0,moldy,113228,Comedy|Romance
4,3,4.0,old,113228,Comedy|Romance


In [33]:
ls = []
for i in range(10):
    ls.append(high_rated_movie[i][0])
df4 = new_df[new_df["movieId"].isin(ls)]

#dropping rating column and we will replace that with avg_rating
df4 = df4.drop("rating",axis=1) #once you drop this column do not run it just comment it


In [34]:
id_rating_pair = [(x,y) for x,y in avg_rating.items() if x in ls]

new_rating = []
for i in range(len(id_rating_pair)):
    new_rating.append(id_rating_pair[i][1])


In [35]:
final_df = df4.sort_values(by="movieId")
final_df["avg_rating"] = new_rating
final_df

,movieId,tag,imdbId,genres,avg_rating
249016,633,NaN,112844,Comedy,5.0
129864,3086,NaN,24852,Children|Comedy|Fantasy|Musical,5.0
271220,3379,NaN,53137,Drama,4.5
153010,3567,NaN,180837,Comedy|Drama|Romance,5.0
249685,3678,NaN,48347,Drama,5.0
246201,3939,NaN,93996,Horror,5.0
246202,3940,NaN,100639,Horror,5.0
246203,3941,NaN,91990,Horror,5.0
246204,3942,NaN,100663,Horror,5.0
189301,3951,In Netflix queue,202641,Drama,5.0


-------